In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from ostrin_animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))
scatterBase = px.scatter(df, x="breed", y="age_upon_outcome_in_weeks")

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.head = [
    html.Link(
        href='https://learn.snhu.edu/content/enforced/636504-CS-340-T3238-OL-TRAD-UG.21EW3/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=0DXGgGQieixrBGpSOwQkxolrU&ou=636504',
        rel='icon'
    ),
]

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(html.Img(src='https://learn.snhu.edu/content/enforced/636504-CS-340-T3238-OL-TRAD-UG.21EW3/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=0DXGgGQieixrBGpSOwQkxolrU&ou=636504', 
                      style={'height':'10%', 'width':'10%'})),
    html.Center(
        html.B(html.H1('SNHU CS-340 Final Dashboard'))
    ),
    html.Hr(),
    dcc.RadioItems(
        id='radiobtns-id',
        options=[
            {'label': 'Water Rescue  ', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue  ', 'value': 'MR'},
            {'label': 'Disaster Rescue or Individual Tracking  ', 'value': 'DR'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        labelStyle={'display': 'inline-block'}
    ),  
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        filter_action="native",
        sort_action="native",
        page_current= 0,
        page_size= 20,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    
    html.Div([
        html.Div([
            dcc.Graph(id="scatter-chart", figure=scatterBase)
        ], style={'width': '46%', 'display': 'inline-block'}),
        html.Div([
            dl.Map(id="map-id", center=[30.75,-97.48], zoom=10)
        ], style={'width': '44%', 'height': '50vh', 'display': 'inline-block'})
    ]),
    html.H2('Daniel Ostrin - SNHU CS-340 Final Dashboard')
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to change grid based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('radiobtns-id', 'value')]
)
def datatable_filter_update(value):
    if value == "WR":
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland" ] }, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks":{ "$gte":26, "$lte":156 } }))
    elif value == "MR":
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler" ] }, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{ "$gte":26, "$lte":156 } }))
    elif value == "DR":
        df = pd.DataFrame.from_records(shelter.read({ "breed": { "$in": [ "Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler" ] }, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{ "$gte":20, "$lte":300 } }))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('rows')


# Callback for filter
@app.callback(
    Output("scatter-chart", "figure"),
    [Input('datatable-id', "data"), 
     Input('radiobtns-id', 'value')])
def interactive_filter_data(viewData, radioValue):
    dff = pd.DataFrame.from_dict(viewData)
    # Generate Chart
    fig = px.scatter(dff, x="breed", y="age_upon_outcome_in_weeks")
    return fig

# Callback for map updates
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
    ]
    


app